# Sales Product Data Analysis ~ (R) ~



The sales product data between 2019-2020.

Key finding in this analysis:

- Total number of sales by year, month, day and hour
- Total number of ordered quantity by city
- Total number of ordered quantity by products
- The probability of customers ordering specific products

This questions would be answer:

- What was the best Year for sales? How much was earned that Year?
- What was the best month for sales? How much was earned that month?
- What City had the highest number of sales?
- What time should we display adverstisement to maximize likelihood of customer's buying product?
- What product sold the most?
- What products are most often sold together?
- How much probability for next people will ordered USB-C Charging Cable, iPhone, Google Phone and Wired Headphones?

In [ ]:
#Import libraries

library(tidyverse)   


In [ ]:
#Read data

jan <- read_csv("/kaggle/input/sales-product-data/Sales_January_2019.csv")
feb <- read_csv("/kaggle/input/sales-product-data/Sales_February_2019.csv")
mar <- read_csv("/kaggle/input/sales-product-data/Sales_March_2019.csv")
apr <- read_csv("/kaggle/input/sales-product-data/Sales_April_2019.csv")
may <- read_csv("/kaggle/input/sales-product-data/Sales_May_2019.csv")
jun <- read_csv("/kaggle/input/sales-product-data/Sales_June_2019.csv")
jul <- read_csv("/kaggle/input/sales-product-data/Sales_July_2019.csv")
aug <- read_csv("/kaggle/input/sales-product-data/Sales_August_2019.csv")
sep <- read_csv("/kaggle/input/sales-product-data/Sales_September_2019.csv")
oct <- read_csv("/kaggle/input/sales-product-data/Sales_October_2019.csv")
nov <- read_csv("/kaggle/input/sales-product-data/Sales_November_2019.csv")
dec <- read_csv("/kaggle/input/sales-product-data/Sales_December_2019.csv")

In [ ]:
#Merge the 12 months of sales data into a single CSV file

all_data <- bind_rows(jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec)

In [ ]:
#Getting to know the data

head(all_data)

In [ ]:
#Check the structure of the data

str(all_data)

In [ ]:
#Explore the data

summary(all_data)

# Data cleansing

In [ ]:
# Rename column  (Optional: For easy to reference)

data <- all_data %>%
rename(order_id = "Order ID", 
       quantity_ordered = "Quantity Ordered",
       price_each = "Price Each", 
       order_date = "Order Date",
       purchase_address = "Purchase Address")

colnames(data)

In [ ]:
#Check for missing data

colSums(is.na(data))

In [ ]:
#Another code for check missing data

data %>%
summarise_all(~sum(is.na(.)))


In [ ]:
#Total of missing data

sum(is.na(data))

In [ ]:
#Remove the rows of missing data

data <- data %>% 
drop_na()

In [ ]:
#Check for missing data again

sum(is.na(data))

In [ ]:
#Check for unique data of the "quantity_ordered" column 

unique(data$quantity_ordered)

In [ ]:
#Filter out text data that not related

data <- data %>%
filter(data$quantity_ordered != "Quantity Ordered")

unique(data$quantity_ordered)

In [ ]:
#Check the data type

glimpse(data)

In [ ]:
#Change Data Type for column "quantity_ordered", "price_each" and "order_date"

data$quantity_ordered<- as.numeric(data$quantity_ordered)

data$price_each<- as.numeric(data$price_each)

data$order_date<- as.POSIXct(data$order_date, format = "%m/%d/%y %H:%M" )

In [ ]:
#Check the data again

head(data)

In [ ]:
#Create the New column of "street_name", "city", "state_code" from splitting the "purchase_address" column

data <- data %>%
separate(purchase_address, 
         into = c("street_name", "city", "state_code"), 
         sep = ",", convert = FALSE)

head(data)

In [ ]:
#Create the New column of "Year", "month", "day","dayofweek", "hour" column from "order_date" column

data <- data %>%
mutate(year = format(data$order_date, format = "%Y"),
       month = format(data$order_date, format = "%B"),
       date = format(data$order_date, format = "%d"),
       dayofweek = format(data$order_date, format = "%A"),
       hour = format(data$order_date, format = "%H"))

head(data)

In [ ]:
#Reorder the month name and date name

data$month = factor(data$month, 
                    levels = month.name)
data$dayofweek = factor(data$dayofweek,
                        levels = c("Sunday", 
                                   "Monday", 
                                   "Tuesday", 
                                   "Wednesday", 
                                   "Thursday", 
                                   "Friday", 
                                   "Saturday"))

head(data)

In [ ]:
#Create the "sales" column by multiplying the "quantity_ordered" column with "price_each" column

data <- data %>%
mutate(sales = quantity_ordered * price_each)

head(data)

# Data visualisation

# Total number of sales by year, month, day and hour

In [ ]:
df_year <- data %>%
group_by(year)%>%
summarize(total_sales = sum(sales))

df_year

> Question: What was the best Year for sales? How much was earned that Year?

> Answer: 2019 was the best year for sales with a total sales of $34,483,365.68

In [ ]:
df_month <- data %>%
group_by(month) %>%
summarize(total_sales = sum(sales)) 

df_month

In [ ]:
options(repr.plot.width = 25, repr.plot.height = 15)
ggplot(data = df_month) + 
geom_bar(mapping = aes(x = month, 
                       y = total_sales,
                       fill = month),
         stat = "identity")+
labs(title = "Total number of sales by months", 
     x = "Months", 
     y = "Total sales") +
theme(axis.text = element_text(size = 20)) +
theme(axis.title = element_text(size = 30), title = element_text(size = 30))

In [ ]:
#Line chart

ggplot(data = df_month) + 
geom_line(mapping = aes(x = month,
                        y = total_sales,
                        group = 1))+
labs(title = "Trend in the Monthly sales", 
     x = "Months", 
     y = "Total sales") +
theme(axis.text = element_text(size = 20)) +
theme(axis.title = element_text(size = 30), title = element_text(size = 30))

> Question : What was the best month for sales? How much was earned that month?
> 
> Answer : Sales was best in December with total sales of $4,613,443. This can be due to the fact that December is a celebration sesaon as people tends to buy gifts for themselves and for others.
> 
> Advertisements can be put out within April, September and Decemember to maximize likelihood of customer's buying product all year round.
> 
> 

In [ ]:
df_day <- data %>%
group_by(dayofweek) %>%
summarize(total_sales = sum(sales)) 

df_day

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 9)
ggplot(data = df_day) + 
geom_line(mapping = aes(x = dayofweek,
                        y = total_sales,
                        group = 1))+
labs(title = "Trend in the Daily sales", 
     x = "Days", 
     y = "Total sales") +
theme(axis.text = element_text(size = 20)) +
theme(axis.title = element_text(size = 30), title = element_text(size = 30))

> Tuesdays were found to be the most profitable day for advertising, generating 5,087,957 USD, while Thursdays had the lowest number of sales(4,839,465 USD).

In [ ]:
df_hour <- data %>%
group_by(hour) %>%
summarize(total_sales = sum(sales))

df_hour

In [ ]:
ggplot(data = df_hour) + 
geom_line(mapping = aes(x = hour,
                        y = total_sales,
                        group = 1))+
labs(title = "Trend in the hourly sales", 
     x = "Hours", 
     y = "Total sales") +
theme(axis.text = element_text(size = 20)) +
theme(axis.title = element_text(size = 30), title = element_text(size = 30))

> Question : What time should we display adverstisement to maximize likelihood of customer's buying products?
> 
> Answer : The best times for advertising were between 10:00-12:00 and 18:00-20:00, when the company typically had the highest sales.

In [ ]:
#Try to use function: facet_wrap(~ dayofweek)

options(repr.plot.width = 18, repr.plot.height = 12)

ggplot(data = data)+
geom_bar(mapping = aes(y = sales, 
                       x = hour,
                       fill = dayofweek),
         stat = "identity",
         show.legend = FALSE)+
facet_wrap(~ dayofweek)+  

labs(title = "Total number of sales by hour in each day", 
     x = "Hours", 
     y = "Total number of sales") +
theme(axis.text = element_text(size = 10)) +
theme(axis.title = element_text(size = 20), 
      title = element_text(size = 25))

> I tried to use facet_wrap function to see what the result is.This chart compare total number of sales by hour in each day of week. 

> For example, On Saturday the best time for advertising were around 11:00-14:00 and 18:00-20:00 

In [ ]:
days_list = c("Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday")
colors_list = c("red", "yellow", "pink", "green", "orange", "blue", "purple")
options(repr.plot.width = 17, repr.plot.height = 8)

data %>% 
select(hour,dayofweek, sales) %>%
group_by(dayofweek, hour) %>%
summarize(sales = sum(sales))%>%
ggplot(aes(x=hour, y=sales, group=dayofweek)) +
geom_line(aes(color=dayofweek)) +
scale_color_manual(name='Day of Week', 
                   labels=days_list, 
                   values=colors_list)+
labs(title = "Total number of sales by hour in each day", 
     x = "Hours", 
     y = "Total number of sales") +
theme(axis.text = element_text(size = 20)) +
theme(axis.title = element_text(size = 30), 
      title = element_text(size = 30))

> From the result, the times for highest sales were during 10:00-12:00 and 18:00-20:00 regardless of what day it is.

In [ ]:
#Try to use function : facet_wrap(~ month)

options(repr.plot.width = 18, repr.plot.height = 12)
ggplot(data = data)+
geom_bar(mapping = aes(y = sales, 
                       x = dayofweek,
                       fill = dayofweek),
         stat = "identity",
         show.legend = FALSE)+
facet_wrap(~ month)+

labs(title = "Total number of sales by day in each month", 
     x = "Day of week", 
     y = "Total number of sales") +
theme(axis.title = element_text(size = 20), 
      title = element_text(size = 30))

> I tried to use facet_wrap function to see what the result is. This chart to compare total number of sales by day of week in each month. 

> For example: In December, the highest sales were on Sunday, Monday and Tuesday. 

In [ ]:
days_list = c("Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday")
colors_list = c("red", "yellow", "pink", "green", "orange", "blue", "purple")
options(repr.plot.width = 20, repr.plot.height = 9)

data %>% 
select(month,dayofweek, sales) %>%
group_by(dayofweek, month) %>%
summarize(sales = sum(sales))%>%
ggplot(aes(x=month, y=sales, group=dayofweek)) +
geom_line(aes(color=dayofweek)) +
scale_color_manual(name="Day of Week", 
                   labels=days_list, 
                   values=colors_list) +
labs(title = "Total number of sales by months in each day", 
     x = "Months", 
     y = "Total number of sales") +
theme(axis.text = element_text(size = 15)) +
theme(axis.title = element_text(size = 30), title = element_text(size = 30))

> Trend in monthly sales compare between day of week. 

> In December, the highest number of sales were on Sunday, Monday and Tuesday. 

> In April, the highest number of sales were on Tuesday. 

# Total number of ordered quantity by city

In [ ]:
#Find the Quantity ordered of each city

df_city <- data %>%
group_by(city)%>%
summarize(total_ordered = sum(quantity_ordered))%>%
arrange(desc(total_ordered))

df_city

In [ ]:
ggplot(data = df_city)+
geom_bar(mapping = aes(x = total_ordered, 
                       y = reorder(city,total_ordered),
                       fill = city),
         stat = "identity",
         show.legend = FALSE)+ 
geom_text(aes(x = total_ordered, 
              y = city,
              label = total_ordered),
          hjust = -0.05, nudge_x = 0.5, size = 8, angle = 0) +
labs(title ="Total number of ordered quantity by city", 
     y = "City", 
     x = "Total ordered quantity")+
scale_x_continuous(labels = scales::comma, 
                     limit = c(0, 60000)) + 
theme_minimal() +
theme(axis.text = element_text(size = 20)) +
theme(axis.title = element_text(size = 30), title = element_text(size = 30))

> Question : What City had the highest number of sales?
> 
> Answer : The city that has the most total quantity sales are San Francisco (50,239), Los Angeles (33,289) and New York City (27,932)

# Total number of ordered quantity by products

In [ ]:
#Find the Quantity ordered of each product

df_product <- data %>%
group_by(Product)%>%
summarize(total_ordered = sum(quantity_ordered))%>%
arrange(desc(total_ordered))

df_product

In [ ]:
head(df_product,1)

> Question : What products sell the most?
> 
> Answer : The best-selling product is AAA Batteries (4-pack) with total quantity sold of 31,017.

In [ ]:
ggplot(data = df_product[1:10,])+
geom_bar(mapping = aes(x = total_ordered,
                       y = reorder(Product,total_ordered),
                       fill = Product),
         stat = "identity",
         show.legend = FALSE)+ 
geom_text(aes(x = total_ordered, 
              y = Product,
              label = total_ordered),
          hjust = -0.05, nudge_x = 0.5, size = 8, angle = 0) +
labs(title ="Top 10 total number of ordered quantity by products", 
     x = "Total ordered quantity", 
     y = "Products")+
scale_x_continuous(labels = scales::comma, 
                     limit = c(0, 35000)) + 
theme_minimal() +
theme(axis.text = element_text(size = 15)) +
theme(axis.title = element_text(size = 25), title = element_text(size = 25))

> Question : What products sell the most?
> 
> Answer : The best-selling product is AAA Batteries (4-pack) with total quantity sold of 31,017.

# What products are most often sold together?

In [ ]:
#Create a new column that join product with the same order ID

df_order <- data %>% 
group_by(order_id) %>%
mutate(product_sold = paste(Product, collapse = " , ")) %>% 
select(order_id, product_sold) 

head(df_order)

In [ ]:
#Check for joined product

df_order %>% filter(grepl(",", product_sold))

In [ ]:
# Remove duplicate

df_order <- df_order[!duplicated(df_order), ]

head(df_order)

In [ ]:
df_order %>% filter(grepl(",", product_sold))

In [ ]:
#Create Combination probability and then count of each probability

counter <- c()

for(product_list in df_order$product_sold) {
    if(grepl(",", product_list)) {
        combination = combn(strsplit(product_list, ",")[[1]], 2)
        for(i in 1:dim(combination)[2]) {
            two_products = paste(sort(combination[,i]), collapse = ",")
            if(two_products %in% names(counter)) {
                counter[two_products] = counter[two_products] + 1   
            } else {
                counter[two_products] = 1
            }
        }    
    }
}

In [ ]:
#Create a new data frame

df_product_group <- data.frame(counter) %>%
arrange(desc(counter))

head(df_product_group)

In [ ]:
#Add index of the column

df_product_group <- cbind(product_together = rownames(df_product_group), df_product_group)
rownames(df_product_group) <- 1:nrow(df_product_group)

head(df_product_group)

In [ ]:
ggplot(data = df_product_group[1:10,])+
geom_bar(mapping = aes(x = counter,
                       y = reorder(product_together,counter),
                      fill = product_together),
         stat = "identity",
         show.legend = FALSE)+ 
geom_text(aes(x = counter, 
              y = product_together,
             label = counter),
          hjust = -0.05, nudge_x = 0.5, size = 8, angle = 0) +
labs(title ="Top 10 total number of ordered quantity by products sold together", 
     x = "Total ordered quantity", 
     y = "Products sold together") +
scale_x_continuous(labels = scales::comma, 
                   limit = c(0, 1000)) + 
theme_minimal() +
theme(axis.text = element_text(size = 15)) +
theme(axis.title = element_text(size = 25), title = element_text(size = 25))

> Question : What products are most often sold together?
> 
> Answer : Products that are often sold together are Lightning Charging Cable and iPhone which sold 886 orders, and USB-C Charging Cable,Google Phone ranked second with 859 orders, and the third most are Wired Headphones and iPhone 443 orders.

# The probability of customers ordering specific products

In [ ]:
#We will start by looking for the probability of each product being picked then filter for each product.

prob_data <- data %>%
count(Product) %>%
mutate(prob = n / sum(n) *100)

In [ ]:
prob_data %>%
filter(Product == "USB-C Charging Cable")

> Question : How much probability for next people will ordered USB-C Charging Cable?
> 
> Answer : The probability that next people will ordered USB-C Charging Cable is 11.77%

In [ ]:
prob_data %>%
filter(Product == "iPhone")

> Question : How much probability for next people will ordered iPhone?
> 
> Answer : The probability that next people will ordered iphone is 3.67%

In [ ]:
prob_data %>%
filter(Product == "Google Phone")

> Question : How much probability for next people will ordered Google Phone?
> 
> Answer : The probability that next people will ordered Google phone is 2.97%

In [ ]:
prob_data %>%
filter(Product == "Wired Headphones")

> Question : How much probability other peoples will ordered Wired Headphones?
> 
> Answer : The probability that next people will order Wired Headphones is 10.15%

In [ ]:
#Export data to csv file

write_csv(data,"sales_data.csv")
write_csv(df_product_group,"product_group_data.csv")

**Conclusion**

- The city that has the most total quantity sales are San Francisco (50,239), Los Angeles (33,289) and New York City (27,932)

- To maximize the company's advertising efforts, it is recommended to advertise in December, October, and April, as the company generated the most orders during these months (4,613,443 USD, 3,736,726 USD, and 3,390,670 USD, respectively).

- Tuesdays were found to be the most profitable day for advertising, generating 5,087,957 USD, while Thursdays had the lowest number of sales(4,839,465 USD).

- The best times for advertising were between 10:00-12:00 and 18:00-20:00, when the company typically had the highest sales.

- Top 5 Best selling product are AAA Batteries (4-pack)(31,017), AA Batteries (4-pack)(27,635), USB-C Charging Cable (23,975), Lightning Charging Cable(23,217), and Wired Headphones (20,557).

- Products that are often sold together are Lightning Charging Cable and iPhone which sold 886 orders, and USB-C Charging Cable,Google Phone ranked second with 859 orders, and the third most are Wired Headphones and iPhone 443 orders.

- The probability of customers ordering specific products compared to all other products was calculated as following: 
       * USB-C Charging Cable: 11.77% 
       * iPhone: 3.67% 
       * Google Phone: 2.97% 
       * Wired Headphones: 10.15%